In [1]:
import qiskit
import qiskit_nature
import qiskit_aer

print(f"Qiskit version: {qiskit.__version__}")
print(f"Qiskit Nature version: {qiskit_nature.__version__}")
print(f"Qiskit Aer version: {qiskit_aer.__version__}")


Qiskit version: 1.4.3
Qiskit Nature version: 0.7.2
Qiskit Aer version: 0.17.1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit

from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo as Molecule
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import AerSimulator, Aer
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import Estimator




qiskit_nature.settings.use_pauli_sum_op = False


In [3]:
def create_nh3_driver(bond_length=1.012, bond_angle=107.8, basis='sto3g'):
    """
    Create a parametric NH3 molecule driver
    
    Parameters:
    bond_length: N-H bond length in Angstrom (default: 1.012) (can vary between 0.9 and 1.2)
    bond_angle: H-N-H bond angle in degrees (default: 107.8)
    basis: basis set (default: 'sto3g')
    """
    # Convert angle to radians
    angle_rad = np.deg2rad(bond_angle)
    
    # Calculate H atom positions in pyramidal geometry
    # Place N at origin, H atoms in pyramidal structure
    h1_x = bond_length * np.sin(angle_rad / 2)
    h1_y = bond_length * np.cos(angle_rad / 2)
    h1_z = 0.0
    
    h2_x = -bond_length * np.sin(angle_rad / 2)
    h2_y = bond_length * np.cos(angle_rad / 2)
    h2_z = 0.0
    
    h3_x = 0.0
    h3_y = -bond_length
    h3_z = 0.0
    
    # Create atom string for PySCF
    atom_string = f"N 0 0 0; H {h1_x:.6f} {h1_y:.6f} {h1_z:.6f}; H {h2_x:.6f} {h2_y:.6f} {h2_z:.6f}; H {h3_x:.6f} {h3_y:.6f} {h3_z:.6f}"
    
    driver = PySCFDriver(
        atom=atom_string,
        basis=basis,
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
    
    return driver

def get_qubit_op(bond_length=1.012):
    """
    Get the qubit operator for NH3 molecule at given bond length
    
    Parameters:
    bond_length: N-H bond length in Angstrom (default: 1.012)
    
    Returns:
    tuple: (qubit_op, num_particles, num_spatial_orbitals, problem, mapper)
    """
    # Create driver with given bond length
    driver = create_nh3_driver(bond_length=bond_length)
    problem = FreezeCoreTransformer(freeze_core=True).transform(driver.run())
    
    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals
    
    mapper = ParityMapper(num_particles=problem.num_particles,)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    
    return (qubit_op, num_particles, num_spatial_orbitals, problem, mapper)

In [4]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

In [5]:
distances = np.arange(0.9, 1.2, 0.02)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=10)
noiseless_simulator = Estimator(approximation=True)

In [6]:
for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(bond_length=dist)
    
    exact_result = exact_solver(qubit_op, problem)
    exact_energies.append(exact_result.total_energies[0].real)
    
    init_state = HartreeFock(
        num_spatial_orbitals=num_spatial_orbitals,
        num_particles=num_particles,
        qubit_mapper=mapper
    )
    
    ansatz = UCCSD(
        num_spatial_orbitals=num_spatial_orbitals,
        num_particles=num_particles,
        initial_state=init_state,
        qubit_mapper=mapper
    )
    
    vqe = VQE(
        estimator=noiseless_simulator,
        ansatz=ansatz,
        optimizer=optimizer,
        initial_point=[0] * ansatz.num_parameters
    )
    
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    
    print(f"Distance: {dist:.2f} - Exact Energy: {exact_result.total_energies[0].real:.6f}, VQE Energy: {vqe_result:.6f}")

Distance: 0.90 - Exact Energy: -55.434841, VQE Energy: -55.434661


KeyboardInterrupt: 

In [ ]:
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.xlabel("Atomic distance (Angstrom)")
plt.ylabel("Energy")
plt.legend()
plt.show()